In [1]:
import sys

sys.path.append('../aml_pipeline/chuck_caption_component/')
# enhanced_doc_analyzer_component/enhanced_document_analyzer


from openai import AzureOpenAI


from src import DocumentProcessor

Encoded tokens: [9906, 220, 14957, 0]
Decoded text: Hello world!


In [2]:
# Example of registering the component in a workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Get workspace
ml_client = MLClient.from_config(
    credential=DefaultAzureCredential()
)

Found the config file in: /config.json


In [3]:

from azureml.core import Workspace, Dataset, Datastore

subscription_id = 'f804f2da-c27b-45ac-bf80-16d4d331776d'
resource_group = 'rg-acadocser-dev-04'
workspace_name = 'mlw-acadocser-dev-04'

workspace = Workspace(subscription_id, resource_group, workspace_name)

# datastore = Datastore.get(workspace, "workspaceblobstore")
# dataset = Dataset.File.from_files(path=(datastore, 'azureml/311471de-f1cf-48e9-b533-042d0f8b6eab'))
# dataset.download(target_path='.', overwrite=True)

In [4]:
import dotenv
import os
import pandas as pd
from azureml.rag.utils.connections import get_connection_by_id_v2

aoai_connection_name = "open_ai_connection" 
aoai_connection_id = ml_client.connections.get(aoai_connection_name).id
# Get connection details
connection = get_connection_by_id_v2(aoai_connection_id)

# Access properties using object methods/attributes
endpoint = connection.endpoint
api_key = connection.api_key

azure_openai_endpoint = endpoint
azure_openai_key = api_key

# Initialize the OpenAI client
openai_vision_client = AzureOpenAI(
    azure_endpoint=azure_openai_endpoint, 
    api_key=azure_openai_key, 
    api_version='2023-03-15-preview'
    )


Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [5]:
# Initialize DocumentProcessor instance locally (minimal setup)
processor = DocumentProcessor(
    input_folder="output_dir/",
    output_folder="temp",  # Temporary placeholder
    openai_client=openai_vision_client,
    vision_deployment_name="gpt-4o",
    max_chunk_length=512
)


stats, chunks_df = processor.process()

2025-03-05 09:48:42,288 - src.document_processor - INFO - Input folder: output_dir
2025-03-05 09:48:42,290 - src.document_processor - INFO - Output folder: temp
2025-03-05 09:48:42,429 - src.document_processor - INFO - Successfully initialized tiktoken tokenizer
2025-03-05 09:48:42,430 - src.document_processor - INFO - Processing CSV file: output_dir/combined_elements_data.csv
2025-03-05 09:48:42,441 - src.document_processor - INFO - Generating summaries in parallel...
2025-03-05 09:48:42,444 - src.document_processor - INFO - output_dir/1-s2.0-S0927796X2030053X-am/1-s2.0-S0927796X2030053X-am_analysis.md
2025-03-05 09:48:42,445 - src.document_processor - INFO - output_dir/2412.20995v1/2412.20995v1_analysis.md
2025-03-05 09:48:42,447 - src.document_processor - INFO - md text: 
## Page 1

KARPA: A Training-free Method of Adapting Knowledge Graph as References for Large Langua...
2025-03-05 09:48:42,452 - src.document_processor - INFO - md text: 
## Page 1

Polymer Informatics: Current Sta

In [6]:
stats

{'processed': 653,
 'errors': 0,
 'chunks': 826,
 'input_file': 'output_dir/combined_elements_data.csv',
 'total_rows': 653,
 'timestamp': '2025-03-05T07:43:57.413876',
 'completion_status': 'success'}

In [7]:
chunks_df

'temp/processed_chunks.jsonl'

In [8]:
from retriever_utils import ImageCaptionViewer

from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
import pandas as pd

azure_search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
azure_search_key = os.getenv("AZURE_SEARCH_KEY")
index_name="myindex"


# Initialize your search client
search_client = SearchClient(
    endpoint=azure_search_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(azure_search_key)
)

# Get image chunks
results = search_client.search(
    search_text="*",
    filter="type eq 'image'",
    select=["id", "pdf_file", "page", "bounding_box", "text", "image_path", 
           "role", "confidence", "source"],
    top=1000
)

# Convert to DataFrame
chunks = []
for result in results:
    chunks.append(dict(result))
df = pd.DataFrame(chunks)

# # Create and display the viewer
viewer = ImageCaptionViewer(df)
viewer.display()

ModuleNotFoundError: No module named 'ipywidgets'

In [ ]:
df.columns


Index(['source', 'bounding_box', 'text', 'id', 'role', 'image_path',
       'confidence', 'page', 'pdf_file', '@search.score',
       '@search.reranker_score', '@search.highlights', '@search.captions'],
      dtype='object')

In [ ]:
for result in results:
    print(result)